<a href="https://colab.research.google.com/github/edinaldoab/challenge_data_science_2/blob/main/notebooks/semanas_03_e_04_recomendador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semanas 03 e 04

Durante as semanas 3 e 4 vamos focar em melhorar o sistema de recomendação da InsightPlaces.
Nosso sistema não está recebendo os clicks que esperava apenas recomendando imoveis das mesmas regiões e na mesma faixa de preço, por isso como parte do time de Data Science precisamos criar uma nova mecânica de recomendação de imóveis.

O objetivo é criar um sistema de recomendação de imóveis baseado em similaridade de características. A base de dados utilizada será a base de dados tratada na semana 2, após transformar as variáveis categóricas em variáveis binárias e antes de aplicarmos a vetorização.

# 1. Preparação do Ambiente

## a. Instalação das dependências

In [1]:
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

## b. Inicialização da SparkSession

In [3]:
import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("recomendador") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

## c. Montagem do Drive

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## d. Inicializando a  UI do Spark 

In [6]:
!wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [7]:
with open('/content/drive/MyDrive/Colab Notebooks/projeto_data_science_imobni/dados/authTokenngrok.txt', 'r') as file:
  content = file.read()
  file.close()

my_authtoken = str(content)

In [8]:
get_ipython().system_raw('./ngrok authtoken ' + my_authtoken)
get_ipython().system_raw('./ngrok http 4050 &')

!curl -s http://localhost:4040/api/tunnels

{"tunnels":[],"uri":"/api/tunnels"}


In [9]:
spark

Carregamento dos dados:

# 2. Carregamento e tratamento dos dados

## a. Leitura do dataset

In [10]:
path='/content/drive/MyDrive/Colab Notebooks/projeto_data_science_imobni/dataset_ml_regressao'

data = spark.read.parquet(path)

In [11]:
data.show(5)

+--------------------+-----+---------+---------+-------+------+----+---------------+----------+-------+---------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|                  id|andar|area_util|banheiros|quartos|suites|vaga|         bairro|condominio|   iptu|    valor|zona_central|zona_norte|zona_oeste|zona_sul|Academia|animais_permitidos|Churrasqueira|condominio_fechado|Elevador|Piscina|Playground|portaria_24h|portao_eletronico|salao_de_festas|
+--------------------+-----+---------+---------+-------+------+----+---------------+----------+-------+---------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|00790b85-56e3-43a...|   11|    166.0|        5|      4|     4|   2|    Jacarepaguá|    2100.0| 4600.0|1750000.0|     

In [12]:
data.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: integer (nullable = true)
 |-- area_util: double (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- vaga: integer (nullable = true)
 |-- bairro: string (nullable = true)
 |-- condominio: double (nullable = true)
 |-- iptu: double (nullable = true)
 |-- valor: double (nullable = true)
 |-- zona_central: integer (nullable = true)
 |-- zona_norte: integer (nullable = true)
 |-- zona_oeste: integer (nullable = true)
 |-- zona_sul: integer (nullable = true)
 |-- Academia: integer (nullable = true)
 |-- animais_permitidos: integer (nullable = true)
 |-- Churrasqueira: integer (nullable = true)
 |-- condominio_fechado: integer (nullable = true)
 |-- Elevador: integer (nullable = true)
 |-- Piscina: integer (nullable = true)
 |-- Playground: integer (nullable = true)
 |-- portaria_24h: integer (nullable = true)
 |-- portao_eletronico: integer (nullable

In [14]:
print(f'Quantidade de linhas: {data.count()}')
print(f'Quantidade de colunas: {len(data.columns)}')

Quantidade de linhas: 66551
Quantidade de colunas: 25


Checagem de nulos: 

In [15]:
from pyspark.sql import functions as f

In [18]:
data.select([f.count(f.when(f.isnull(c), 1)).alias(c) for c in data.columns]).show()

+---+-----+---------+---------+-------+------+----+------+----------+----+-----+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
| id|andar|area_util|banheiros|quartos|suites|vaga|bairro|condominio|iptu|valor|zona_central|zona_norte|zona_oeste|zona_sul|Academia|animais_permitidos|Churrasqueira|condominio_fechado|Elevador|Piscina|Playground|portaria_24h|portao_eletronico|salao_de_festas|
+---+-----+---------+---------+-------+------+----+------+----------+----+-----+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|  0|    0|        0|        0|      0|     0|   0|     0|         0|   0|    0|           0|         0|         0|       0|       0|                 0|            0|                 0|       0|      0|         0|    

## b. Seleção de features

Aqui devem ser removidas as colunas que possuem apenas um valor atribuído, de acordo com o que foi elaborado na primeira semana. Esta etapa é muito importante para se reduzir a complexidade dos dados e aumentar a velocidade de ajuste do modelo.

In [ ]:
data\
    .select('tipo')\
    .distinct()\
    .show(5)

+-----+
| tipo|
+-----+
|Venda|
+-----+



In [ ]:
data\
    .select('tipo_anuncio')\
    .distinct()\
    .show(5)

+------------+
|tipo_anuncio|
+------------+
|       Usado|
+------------+



In [ ]:
data\
    .select('tipo_unidade')\
    .distinct()\
    .show(5)

+------------+
|tipo_unidade|
+------------+
| Apartamento|
+------------+



In [ ]:
data\
    .select('tipo_uso')\
    .distinct()\
    .show(5)

+-----------+
|   tipo_uso|
+-----------+
|Residencial|
+-----------+



As colunas `area_total` e `area_util` têm um potencial maior de aderência ao modelo. Assim, analisa-se:

In [ ]:
data\
    .select('area_total','area_util')\
    .show(5)

+----------+---------+
|area_total|area_util|
+----------+---------+
|       166|      166|
|       640|      640|
|        50|       50|
|       160|      160|
|        52|       52|
+----------+---------+
only showing top 5 rows

